<a href="https://colab.research.google.com/github/LorenzoKaufmann/crypto_forecast/blob/main/Collect_Crypto_Currency_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-dotenv

In [3]:
!pip install cryptocompare plotly

  Preparing metadata (setup.py) ... done
  Created wheel for cryptocompare: filename=cryptocompare-0.7.6-py3-none-any.whl size=6291 sha256=ef8de6de69f88c1f500b285dda705379cd9c158c4c9428e8b67afa4f34174ee9
  Stored in directory: /root/.cache/pip/wheels/ac/d9/d3/45937831310f976e534a0a1dd2566a04e9d17ecd42a6932f31
Successfully built cryptocompare


In [10]:
# from pycoingecko import CoinGeckoAPI
import cryptocompare
import pandas as pd
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from google.colab import  drive
import os
import numpy as np
from itertools import islice
import dotenv
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots

drive.mount('/drive')
folder_path = '/drive/My Drive/Colab Notebooks/Crypto Project/Data/cryptocompare'

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [5]:
plot_results = True

In [6]:
cryptocurrency_release_dates = {
    "Bitcoin (BTC)": "2012-01-01",
    "Ethereum (ETH)": "2015-07-30",
    "Tether (USDT)": "2014-10-06",
    "Binance Coin (BNB)": "2017-07-25",
    "USD Coin (USDC)": "2018-09-26",
    "Ripple (XRP)": "2012-01-01",
    "Solana (SOL)": "2020-03-01",
    "Cardano (ADA)": "2017-09-29",
    "Dogecoin (DOGE)": "2013-12-06",
    "Toncoin (TON)": "2021-08-01",
    "Polkadot (DOT)": "2020-05-26",
    "Litecoin (LTC)": "2012-01-01",
    "Chainlink (LINK)": "2017-09-19",
    "Uniswap (UNI)": "2020-09-16",
    "Bitcoin Cash (BCH)": "2017-08-01",
    "Stellar (XLM)": "2014-07-31",
    "VeChain (VET)": "2018-08-01",
    "TRON (TRX)": "2017-09-13",
    "Filecoin (FIL)": "2020-10-15",
    "Aave (AAVE)": "2020-10-01"
}

In [7]:
for coin in cryptocurrency_release_dates.keys():
  coin_short = coin.split('(')[1][:-1]
  # print(f"{coin_short.lower()}")

btc
eth
usdt
bnb
usdc
xrp
sol
ada
doge
ton
dot
ltc
link
uni
bch
xlm
vet
trx
fil
aave


# Get historical data with CryptoCompare

---



In [11]:
dotenv.load_dotenv('/drive/My Drive/Colab Notebooks/Crypto Project/.env')
API_KEY = os.getenv('API_KEY_Crypto_Compare')

def get_historical_data(crypto_coin, currency, start_date, end_date):
  url = 'https://min-api.cryptocompare.com/data/v2/histoday'
  if type(start_date) == str:
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

  limit = end_date - start_date
  limit = limit.days
  if limit > 2000:
    print('limit is too high')
    return
  cryptocompare.cryptocompare._set_api_key_parameter(API_KEY)
  try:
    cc = cryptocompare.get_historical_price_day(crypto_coin, currency,
                                                limit=limit, exchange='CCCAGG',
                                                toTs=end_date)
    df_cc = pd.DataFrame(cc)
    df_cc['time'] = pd.to_datetime(df_cc['time'], unit='s')
  except KeyError:
    print('ERROR')
    return
  return df_cc


for coin, release_date in cryptocurrency_release_dates.items():
  coin_name = coin.split('(')[0]
  coin_symbol = coin.split('(')[1][:-1]
  date_range = pd.date_range(start=release_date, end="2024-10-01", freq='1YE')
  print(f'Coin: {coin}')
  for sd, ed in zip(date_range[:-1], date_range[1:]):
    # print(f'Coin: {coin} \n Start: {sd}\n End: {ed}')
    hist_data = get_historical_data(coin_symbol, 'USD', sd, ed)
    if hist_data is None:
      continue
    hist_data = hist_data[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']].iloc[:-1]


    if os.path.exists(folder_path):
      if os.path.exists(f'{folder_path}/{coin_symbol}_file.csv'):
        df_old = pd.read_csv(f'{folder_path}/{coin_symbol}_file.csv', parse_dates=['time'], index_col=0, date_format='%Y-%m-%d')
        last_entry_date = df_old.index[-1]
        new_entry_date = hist_data.index[0]
        hist_data = pd.concat([df_old, hist_data], axis=0)
        hist_data.to_csv(f'{folder_path}/{coin_symbol}_file.csv')
      else:
        hist_data.to_csv(f'{folder_path}/{coin_symbol}_file.csv')

Coin: Bitcoin (BTC)
Coin: Ethereum (ETH)
Coin: Tether (USDT)
Coin: Binance Coin (BNB)
Coin: USD Coin (USDC)
Coin: Ripple (XRP)
Coin: Solana (SOL)
Coin: Cardano (ADA)
Coin: Dogecoin (DOGE)
Coin: Toncoin (TON)
Coin: Polkadot (DOT)
Coin: Litecoin (LTC)
Coin: Chainlink (LINK)
Coin: Uniswap (UNI)
Coin: Bitcoin Cash (BCH)
Coin: Stellar (XLM)
Coin: VeChain (VET)
Coin: TRON (TRX)
Coin: Filecoin (FIL)
Coin: Aave (AAVE)


## Get latest Updates in Crypto Currencies

In [12]:
for coin, release_date in cryptocurrency_release_dates.items():
  coin_name = coin.split('(')[0]
  coin_symbol = coin.split('(')[1][:-1]
  release_date = pd.to_datetime(release_date)
  try:
    df_old = pd.read_csv(f'{folder_path}/{coin_symbol}_file.csv', parse_dates=['time'], index_col=0, date_format='%Y-%m-%d')
    df_old = df_old[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]

  except FileNotFoundError:
    print(f"No file found for {coin_symbol}")
    continue

  if not df_old.empty:
      last_date = df_old['time'].max()
  else:
      last_date = release_date

  # Ensure last_date is at least one day before today
  today = pd.Timestamp.today().normalize()
  last_date = pd.to_datetime(last_date)
  if last_date >= today:
      print(f"No update needed for {coin_symbol}")
  else:
      # Set the end_date for fetching new data (one day before today)
      days_to_fetch = (today - last_date).days + 2 # just to be save
      end_date = datetime.now() - timedelta(days=1)
      last_timestamp = int(last_date.timestamp())

      new_data = cryptocompare.get_historical_price_day(coin_symbol, 'USD',
                                                        limit=days_to_fetch, exchange='CCCAGG',
                                                        toTs=int(end_date.timestamp()))
      df_new = pd.DataFrame(new_data)
      df_new['time'] = pd.to_datetime(df_new['time'], unit='s')
      df_new = df_new[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]

      # Filter new data to ensure it's only after the last date in df_old
      df_new = df_new[df_new['time'] > last_date]

      # Append new data to old data
      df_updated = pd.concat([df_old, df_new], axis=0)

      df_updated.to_csv(f'{folder_path}/{coin_symbol}_file.csv')

      print(f"Data updated for {coin_symbol}")


Data updated for BTC
Data updated for ETH
Data updated for USDT
Data updated for BNB
Data updated for USDC
Data updated for XRP
Data updated for SOL
Data updated for ADA
Data updated for DOGE
Data updated for TON
Data updated for DOT
Data updated for LTC
Data updated for LINK
Data updated for UNI
Data updated for BCH
Data updated for XLM
Data updated for VET
Data updated for TRX
Data updated for FIL
Data updated for AAVE


## Load Data

In [13]:
coin_dict = {}
for coin, release_date in cryptocurrency_release_dates.items():
  coin_name = coin.split('(')[0]
  coin_symbol = coin.split('(')[1][:-1]
  release_date = pd.to_datetime(release_date)
  try:
    currency_df = pd.read_csv(f'{folder_path}/{coin_symbol}_file.csv', parse_dates=['time'], index_col=0, date_format='%Y-%m-%d')
    currency_df = currency_df[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]
  except FileNotFoundError:
    print(f"No file found for {coin_symbol}")
    continue

  coin_dict[coin_name] = currency_df